# Generating `.json` corpus

In [1]:
from prediction_helpers import make_json_from_texts

In [2]:
make_json_from_texts(folder_path="texts"
                     , output_dir="new_content_preprocessed"
                    )

INFO:root:📚 Turning .txt files in texts to .json corpus.
INFO:root:	2 .txt files found in folder:
			Encore_Heidi_09.txt
			Les_Aventures_de_Nono_VII.txt
INFO:root:📨 New .json corpus stored at new_content_preprocessed/new_corpus.json


# Generating `.tsv` files

In [2]:
import sys
sys.path.append('../')

from preprocessing.data_utils import *

In [4]:
expander_args = {"document_maker":spacy_nlp
                 #, "annotated":False
                 , "label_name":"label"
                 , "label_pos":"DS"
                 , "label_neg":"O"
                 , "annotation_scheme":None
                }

In [5]:
generate_tokens_df(raw_data_path="new_content_preprocessed/new_corpus.json"
                   , output_dir="new_content_preprocessed/"
                   , do_split=False
                   ,**expander_args
                  ) 

2it [00:02,  1.36s/it]

Loaded corpus of 2 files.
Saving Dataframe with columns:
 []
 composed of 0 rows at:
new_content_preprocessed/train.tsv

Saving Dataframe with columns:
 []
 composed of 0 rows at:
new_content_preprocessed/val.tsv

Saving Dataframe with columns:
 ['token', 'sentstart', 'token_idx', 'label', 'file']
 composed of 11667 rows at:
new_content_preprocessed/test.tsv



In [6]:
pd.read_csv("new_content_preprocessed/test.tsv", sep="\t")

,token,sentstart,token_idx,label,file
0,La,yes,"(0, 2)",O,Encore_Heidi_09
1,veille,no,"(3, 9)",O,Encore_Heidi_09
2,du,no,"(10, 12)",O,Encore_Heidi_09
3,jour,no,"(13, 17)",O,Encore_Heidi_09
4,où,no,"(18, 20)",O,Encore_Heidi_09
...,...,...,...,...,...
11662,à,no,"(10370, 10371)",O,Les_Aventures_de_Nono_VII
11663,sa,no,"(10372, 10374)",O,Les_Aventures_de_Nono_VII
11664,chambre,no,"(10375, 10382)",O,Les_Aventures_de_Nono_VII
11665,.,no,"(10382, 10383)",O,Les_Aventures_de_Nono_VII


# Run AADS

In [3]:
import sys
sys.path.append('../')

import run_experiments

In [8]:
! python ../run_experiments.py --configs_folder new_content_configs

INFO:root:Parsing all json config files from new_content_configs.
INFO:root:🗣 RUNNING 3 (W)DSR EXPRIMENTS
INFO:root:
+--------------------------------------------------------------------------------------------------+
|                                     LODADED CONFIG 1 — flair                                     |
+--------------------------------------------------------------------------------------------------+
| - model_type                                    flair                                            |
| - model_natural_name                            flair                                            |
+--------------------------------------------------------------------------------------------------+
| - embedding_name                                flair_stacked_fasttext                           |
| - max_len                                       192                                              |
| - batch_len                                     8                        

# Merge results

In [4]:
from prediction_helpers import merge_predictions

In [10]:
merged_df = merge_predictions(methods=["regex", "flair", "transformer"]
                             )

INFO:root:Tokenized text with merged predictions saved at output/merged_predictions.tsv


In [11]:
merged_df

,file,token,sentstart,token_idx,pred_regex,pred_flair,pred_transformer
0,Encore_Heidi_09,La,yes,"(0, 2)",O,O,O
1,Encore_Heidi_09,veille,no,"(3, 9)",O,O,O
2,Encore_Heidi_09,du,no,"(10, 12)",O,O,O
3,Encore_Heidi_09,jour,no,"(13, 17)",O,O,O
4,Encore_Heidi_09,où,no,"(18, 20)",O,O,O
...,...,...,...,...,...,...,...
11662,Les_Aventures_de_Nono_VII,à,no,"(10370, 10371)",O,O,O
11663,Les_Aventures_de_Nono_VII,sa,no,"(10372, 10374)",O,O,O
11664,Les_Aventures_de_Nono_VII,chambre,no,"(10375, 10382)",O,O,O
11665,Les_Aventures_de_Nono_VII,.,no,"(10382, 10383)",O,O,O
